# Generate ROMs from the Object Files Generated by Failed LINKM Build

In [150]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

While the `SAV` files are more or less raw binary output from the Atari MAC65 assembler, `LDA` is an PDP11/RT-11 file format. It's not a raw binary dump, instead the data is stored in chunks each one with with a header and checksum.

A chunk looks something like this (all words are little-endian):
```
Sentinel: 0x0100
Length: 0x0B00
Load Address: 0x0090
Data: 0x0102030405
Checksum: 0x09
```

You keep reading the chunks above until there are none left! See http://www.bitsavers.org/pdf/dec/pdp11/rt11/v5.6_Aug91/AA-PD6PA-TC_RT-11_Volume_and_File_Formats_Manual_Aug91.pdf. So let's start with `TEMPST.LDA` and extract the binary data from it.

## Generate ROMs for Tempest Version 1

A short function to parse LDA files.

In [1]:
word = lambda x: (x).to_bytes(2,"little")
read_word = lambda x: int.from_bytes(x,"little")

def parse_lda_file(filename):
    BLOCK_HEADER = b'\x01\x00'
    LDA_END = 0x0000
    
    al = open(filename,'rb')
    blocks = []
    while True:
        sentinel = al.read(2)
        if sentinel != BLOCK_HEADER:
            raise Exception("Block Header Expected But Got: ", sentinel)

        len_data = read_word(al.read(2)) - 6
        # Have we reached the end of our blocks?
        if len_data == LDA_END:
            break

        addr = read_word(al.read(2))
        data = al.read(len_data)
        checksum = al.read(1)
        blocks += [(hex(addr),addr,len(data),data)]
        
    # Use the addresses given in our datablocks to populate
    # a bytearray of all of the ROM data.
    output_bytes = bytearray(b'\x00' * 0xFFFF)
    for _,addr,len_data,data in blocks:
        output_bytes[addr:addr+len_data] = data

    return output_bytes

Let's use it to parse `TEMPST.LDA`.

In [2]:
output_bytes = parse_lda_file("rk05_tempest_object_files/ALEXEC.LDA")

Now we have the 'raw' bytes for our ROMs in `output_bytes`.

The manifest for Version 2A (alt) in `TEMPST.DOC` tells us what ROM names to use. 


```
Project #:  28903			Project Programmer:  Dave Theurer		Project Leader:  Morgan Hoff

Documentation Disk(s): 36A,36B,36C		Project Name:  Tempest			Date:  12-17-81		Version: 2A (alt)


Verification control filename: 002X2.DAT	Verification file: ALEXEC.LDA


Part #		PCB Loc.	Description		Start Addr	Generic name	Part Size	Bits Used
------		--------	-----------		----------	------------	---------	---------	

136002-138	N/P3		VG EROM			3000		2532		4KX8		7->0				

136002-237	P1		PROGRAM EROM		9000		2532		4KX8		7->0
136002-136	L/M1		PROGRAM EROM		A000		2532		4KX8		7->0
136002-235	J1		PROGRAM EROM		B000		2532		4KX8		7->0
136002-134	F1		PROGRAM EROM		C000		2532		4KX8		7->0
136002-133	D1		PROGRAM EROM		D000		2532		4KX8		7->0
				(ind. self test)


General Information:
--------------------

Assembler used:  MAC65

Main program source files:
	Includes:  ALCOMN, HLL65, COIN65, VGMC, ANVGAN
	Files:  ALWELG,ALSCO2,ALDIS2,ALEXEC,ALSOUN,ALVROM,ALCOIN,ALLANG,ALHAR2,ALTES2,ALEARO,ALVGUT

Picture file:

Linker used:  LINKM

Linker command:  ALEXEC/L,ALEXEC/A=ALWELG,ALSCO2,ALDIS2,ALEXEC,ALSOUN,ALVROM,ALCOIN,ALLANG,ALHAR2,ALTES2,ALEARO,ALVGUT
```


The corresponding entry in Mame's manifest given as 'Rev 3'. It is the 'last' release of Tempest so is given simply as `tempest`. Notice that the addresses for the ROMs are reversed in the Mame entry. `136002-133.d1` is loaded to `0x9000` rather than `0xD000` as specified by the Atari version control doc above.

```cpp
ROM_START( tempest ) /* rev 3 */
	/* Roms are for Tempest Analog Vector-Generator PCB Assembly A037383-03 or A037383-04 */
	ROM_REGION( 0x10000, "maincpu", 0 )
	ROM_LOAD( "136002-133.d1",  0x9000, 0x1000, CRC(1d0cc503) SHA1(7bef95db9b1102d6b1166bda0ccb276ef4cc3764) ) /* 136002-113 + 136002-114 */
	ROM_LOAD( "136002-134.f1",  0xa000, 0x1000, CRC(c88e3524) SHA1(89144baf1efc703b2336774793ce345b37829ee7) ) /* 136002-115 + 136002-316 */
	ROM_LOAD( "136002-235.j1",  0xb000, 0x1000, CRC(a4b2ce3f) SHA1(a5f5fb630a48c5d25346f90d4c13aaa98f60b228) ) /* 136002-217 + 136002-118 */
	ROM_LOAD( "136002-136.lm1", 0xc000, 0x1000, CRC(65a9a9f9) SHA1(73aa7d6f4e7093ccb2d97f6344f354872bcfd72a) ) /* 136002-119 + 136002-120 */
	ROM_LOAD( "136002-237.p1",  0xd000, 0x1000, CRC(de4e9e34) SHA1(04be074e45bf5cd95a852af97cd04e35b7f27fc4) ) /* 136002-121 + 136002-222 */
	ROM_RELOAD(                 0xf000, 0x1000 ) /* for reset/interrupt vectors */

	/* Vector ROM */
	ROM_REGION( 0x1000, "vectorrom", 0 )
	ROM_LOAD( "136002-138.np3", 0x0000, 0x1000, CRC(9995256d) SHA1(2b725ee1a57d423c7d7377a1744f48412e0f2f69) )

	/* AVG PROM */
	ROM_REGION( 0x100, "avg:prom", 0 )
	ROM_LOAD( "136002-125.d7",   0x0000, 0x0100, CRC(5903af03) SHA1(24bc0366f394ad0ec486919212e38be0f08d0239) )

	/* Mathbox PROMs */
	ROM_REGION( 0x20, "user2", 0 )
	ROM_LOAD( "136002-126.a1",   0x0000, 0x0020, CRC(8b04f921) SHA1(317b3397482f13b2d1bc21f296d3b3f9a118787b) )

	ROM_REGION32_BE( 0x400, "user3", 0 )
	ROMX_LOAD( "136002-132.l1", 0, 0x100, CRC(2af82e87) SHA1(3816835a9ccf99a76d246adf204989d9261bb065), ROM_NIBBLE | ROM_SHIFT_NIBBLE_LO | ROM_SKIP(3))
	ROMX_LOAD( "136002-131.k1", 0, 0x100, CRC(b31f6e24) SHA1(ce5f8ca34d06a5cfa0076b47400e61e0130ffe74), ROM_NIBBLE | ROM_SHIFT_NIBBLE_HI | ROM_SKIP(3))
	ROMX_LOAD( "136002-130.j1", 1, 0x100, CRC(8119b847) SHA1(c4fbaedd4ce1ad6a4128cbe902b297743edb606a), ROM_NIBBLE | ROM_SHIFT_NIBBLE_LO | ROM_SKIP(3))
	ROMX_LOAD( "136002-129.h1", 1, 0x100, CRC(09f5a4d5) SHA1(d6f2ac07ca9ee385c08831098b0dcaf56808993b), ROM_NIBBLE | ROM_SHIFT_NIBBLE_HI | ROM_SKIP(3))
	ROMX_LOAD( "136002-128.f1", 2, 0x100, CRC(823b61ae) SHA1(d99a839874b45f64e14dae92a036e47a53705d16), ROM_NIBBLE | ROM_SHIFT_NIBBLE_LO | ROM_SKIP(3))
	ROMX_LOAD( "136002-127.e1", 2, 0x100, CRC(276eadd5) SHA1(55718cd8ec4bcf75076d5ef0ee1ed2551e19d9ba), ROM_NIBBLE | ROM_SHIFT_NIBBLE_HI | ROM_SKIP(3))
ROM_END
```

In [3]:
OUT_DIR="failed_build_roms/tempest/"

open(f"{OUT_DIR}136002-123.np3",'wb').write(bytes(output_bytes[0x3000:0x3800]))
open(f"{OUT_DIR}136002-124.r3",'wb').write(bytes(output_bytes[0x3800:0x4000]))

open(f"{OUT_DIR}136002-138.np3",'wb').write(bytes(output_bytes[0x3000:0x4000]))

# Notice we have the addresses in reverse order, e.g. 136002-237 is written to 0xD000 rather
# than 0x9000.
open(f"{OUT_DIR}136002-237.p1",'wb').write(bytes(output_bytes[0xD000:0xE000]))
open(f"{OUT_DIR}136002-136.lm1",'wb').write(bytes(output_bytes[0xC000:0xD000]))
open(f"{OUT_DIR}136002-235.j1",'wb').write(bytes(output_bytes[0xB000:0xC000]))
open(f"{OUT_DIR}136002-134.f1",'wb').write(bytes(output_bytes[0xA000:0xB000]))
open(f"{OUT_DIR}136002-133.d1",'wb').write(bytes(output_bytes[0x9000:0xA000]))


4096

As with version 1 we need the mathbox and vector generator ROMs.

In [4]:
mathbox_bytes = open("tempest/MBOX.SAV",'rb').read()
open(f"{OUT_DIR}/136002-126.a1",'wb').write(mathbox_bytes[0x8400:0x8420])
# 136002-127.e1 contains the low 4 bits of each byte in 0x6A00:0x6B00 in MBOX.SAV
open(f"{OUT_DIR}/136002-127.e1",'wb').write(bytes([a & 0x0F for a in mathbox_bytes[0x6A00:0x6B00]]))
# 136002-128.f1 contains the high 4 bits (right-shifted) of each byte in 0x6A00:0x6B00 in MBOX.SAV
open(f"{OUT_DIR}/136002-128.f1",'wb').write(bytes([(a & 0xF0) >> 4 for a in mathbox_bytes[0x6A00:0x6B00]]))

# 136002-129.h1 contains the low 4 bits of each byte in 0x6900:0x6A00 in MBOX.SAV
open(f"{OUT_DIR}/136002-129.h1",'wb').write(bytes([a & 0x0F for a in mathbox_bytes[0x6900:0x6A00]]))
# 136002-130.j1 contains the high 4 bits (right-shifted) of each byte in 0x6900:0x6A00 in MBOX.SAV
open(f"{OUT_DIR}/136002-130.j1",'wb').write(bytes([(a & 0xF0) >> 4 for a in mathbox_bytes[0x6900:0x6A00]]))

# 136002-131.k1 contains the low 4 bits of each byte in 0x6800:0x6900 in MBOX.SAV
open(f"{OUT_DIR}/136002-131.k1",'wb').write(bytes([a & 0x0F for a in mathbox_bytes[0x6800:0x6900]]))
# 136002-132.l1 contains the high 4 bits (right-shifted) of each byte in 0x6800:0x6900 in MBOX.SAV
open(f"{OUT_DIR}/136002-132.l1",'wb').write(bytes([(a & 0xF0) >> 4 for a in mathbox_bytes[0x6800:0x6900]]))


256

In [5]:
vecgen_bytes = open("tempest/STATE2.SAV",'rb').read()
open(f"{OUT_DIR}/136002-125.d7",'wb').write(vecgen_bytes[0x1000:0x1100])

256

Now let's see if it works:

In [6]:
!mame -window -rompath ./failed_build_roms/ tempest

Unable to find the BGFX path bgfx, please install it or fix the bgfx_path setting to use the BGFX renderer.
136002-133.d1 WRONG CHECKSUMS:
    EXPECTED: CRC(1d0cc503) SHA1(7bef95db9b1102d6b1166bda0ccb276ef4cc3764)
       FOUND: CRC(b2e2384c) SHA1(076b006521b1f21ef2dba37c65d89f7dec2c94e0)
136002-134.f1 WRONG CHECKSUMS:
    EXPECTED: CRC(c88e3524) SHA1(89144baf1efc703b2336774793ce345b37829ee7)
       FOUND: CRC(1cad96da) SHA1(822d826ea831d99f391d3d66b5557916df2e74cf)
136002-235.j1 WRONG CHECKSUMS:
    EXPECTED: CRC(a4b2ce3f) SHA1(a5f5fb630a48c5d25346f90d4c13aaa98f60b228)
       FOUND: CRC(63ff402b) SHA1(3fb3f1dd0774f183661088383f6af87736d7d2d5)
136002-136.lm1 WRONG CHECKSUMS:
    EXPECTED: CRC(65a9a9f9) SHA1(73aa7d6f4e7093ccb2d97f6344f354872bcfd72a)
       FOUND: CRC(83e56b0e) SHA1(2ef156abbb456bb2a2bdae2a76362fcc53ea571a)
136002-237.p1 WRONG CHECKSUMS:
    EXPECTED: CRC(de4e9e34) SHA1(04be074e45bf5cd95a852af97cd04e35b7f27fc4)
       FOUND: CRC(3bb3714a) SHA1(c83a458e3ea4501dc5e4b4ba28de

No worky!

The first three files are byte for byte identical:

In [9]:
!diff -y -W 150 <(dd if=failed_build_roms/tempest/136002-123.np3|xxd) <(dd if=roms/136002-123.np3|xxd) |grep '  |	'
!diff -y -W 150 <(dd if=failed_build_roms/tempest/136002-124.r3|xxd) <(dd if=roms/136002-124.r3|xxd) |grep '  |	'
!diff -y -W 150 <(dd if=failed_build_roms/tempest/136002-138.np3|xxd) <(dd if=roms/136002-138.np3|xxd) |grep '  |	'

4+0 records in
4+0 records out
2048 bytes (2.0 kB, 2.0 KiB) copied, 2.1438e-05 s, 95.5 MB/s
4+0 records in
4+0 records out
2048 bytes (2.0 kB, 2.0 KiB) copied, 1.6412e-05 s, 125 MB/s
4+0 records in
4+0 records out
2048 bytes (2.0 kB, 2.0 KiB) copied, 4.5882e-05 s, 44.6 MB/s
4+0 records in
4+0 records out
2048 bytes (2.0 kB, 2.0 KiB) copied, 4.706e-05 s, 43.5 MB/s


But it goes south from there:

In [14]:
!diff -y -W 150 <(dd if=failed_build_roms/tempest/136002-133.d1|xxd) <(dd if=roms/136002-133.d1|xxd) |grep '  |	'

8+0 records in
8+0 records out
4096 bytes (4.1 kB, 4.0 KiB) copied, 2.1865e-05 s, 187 MB/s
8+0 records in
8+0 records out
4096 bytes (4.1 kB, 4.0 KiB) copied, 1.9435e-05 s, 211 MB/s
00000030: 9220 4692 209f 9220 ad92 2063 c1a9 ff8d  . F. .. .. c....	  |	00000030: 9220 4692 209f 9220 ad92 206e c1a9 ff8d  . F. .. .. n....
00000140: 89c1 a910 8504 20ad 92ce 0506 101b f8a5  ...... .........	  |	00000140: 96c1 a910 8504 20ad 92ce 0506 101b f8a5  ...... .........
00000160: 0320 f0cc a914 8d05 0620 a8b0 a918 a404  . ....... ......	  |	00000160: 0320 fecc a914 8d05 0620 abb0 a918 a404  . ....... ......
00000190: 8448 adca 6029 0795 4685 9f20 89c1 20c5  .H..`)..F.. .. .	  |	00000190: 8448 adca 6029 0795 4685 9f20 96c1 20c5  .H..`)..F.. .. .
000007b0: 02f0 0320 a7cc a52a 8d00 02a5 2b8d 0102  ... ...*....+...	  |	000007b0: 02f0 0320 b5cc a52a 8d00 02a5 2b8d 0102  ... ...*....+...
00000800: 0130 0160 ad02 02c9 10d0 0320 e0cc ad07  .0.`....... ....	  |	00000800: 0130 0160 ad02 02c9 10d0 0320 eecc a

In [15]:
!diff -y -W 150 <(dd if=failed_build_roms/tempest/136002-134.f1|xxd) <(dd if=roms/136002-134.f1|xxd) |grep '  |	'

8+0 records in
8+0 records out
4096 bytes (4.1 kB, 4.0 KiB) copied, 2.0091e-05 s, 204 MB/s
8+0 records in
8+0 records out
4096 bytes (4.1 kB, 4.0 KiB) copied, 1.9324e-05 s, 212 MB/s
00000210: 99ac 03fe f202 a9c0 999a 0320 e8cc a2ff  ........... ....	  |	00000210: 99ac 03fe f202 a9c0 999a 0320 f6cc a2ff  ........... ....
00000220: a900 852a 852b a901 8529 205e caa6 37bd  ...*.+...) ^..7.	  |	00000220: a900 852a 852b a901 8529 206c caa6 37bd  ...*.+...) l..7.
00000290: c002 a900 9df2 0220 dccc ad02 0220 63a4  ....... ..... c.	  |	00000290: c002 a900 9df2 0220 eacc ad02 0220 63a4  ....... ..... c.
000002f0: ad19 019d a602 20af cce6 a6a0 0088 10d9  ...... .........	  |	000002f0: ad19 019d a602 20bd cce6 a6a0 0088 10d9  ...... .........
00000330: 1869 05aa 205e caa6 3760 a905 2052 a3ce  .i.. ^..7`.. R..	  |	00000330: 1869 05aa 206c caa6 3760 a905 2052 a3ce  .i.. l..7`.. R..
00000350: a901 852c ad02 0285 29ad 0002 852d 20a2  ...,....)....- .	  |	00000350: a901 852c ad02 0285 29ad 0002 852d 2

In [16]:
!diff -y -W 150 <(dd if=failed_build_roms/tempest/136002-235.j1|xxd) <(dd if=roms/136002-235.j1|xxd) |grep '  |	'

8+0 records in
8+0 records out
4096 bytes (4.1 kB, 4.0 KiB) copied, 2.0887e-05 s, 196 MB/s
8+0 records in
8+0 records out
4096 bytes (4.1 kB, 4.0 KiB) copied, 1.906e-05 s, 215 MB/s
00000000: 3abc fe91 c063 b037 c898 2074 afa0 0320  :....c.7.. t... 	  |	00000000: 75df a63a bcfe 91c0 63b0 37c8 9820 77af  u..:....c.7.. w.
00000010: ceb0 200c aba2 baa4 37b9 93b0 1869 ec20  .. .....7....i. 	  |	00000010: a003 20d1 b020 0dab a2ba a437 b996 b018  .. .. .....7....
00000020: 65df a63a 20c3 b020 0cab a2cc a437 b993  e..: .. .....7..	  |	00000020: 69ec 2075 dfa6 3a20 c6b0 200d aba2 cca4  i. u..: .. .....
00000030: b018 6900 2065 dfa6 3abd fe91 20d4 c4c6  ..i. e..:... ...	  |	00000030: 37b9 96b0 1869 0020 75df a63a bdfe 9120  7....i. u..:... 
00000040: 3ac6 3710 a1a9 0085 7320 0cab a21c 2013  :.7.....s .... .	  |	00000040: e1c4 c63a c637 10a1 a900 8573 200d aba2  ...:.7.....s ...
00000050: aba9 04a0 0120 a1df a000 20ce b020 0cab  ..... .... .. ..	  |	00000050: 1c20 14ab a904 a001 20b1 dfa0 0020 d1

Let's compare the contents of the two LDA files in toto.

In [26]:
failed_build = parse_lda_file("rk05_tempest_object_files/ALEXEC.LDA")
real_build = parse_lda_file("tempest/ALEXEC.LDA")
open("failed_build",'wb').write(failed_build)
open("real_build",'wb').write(real_build)
!diff -y -W 150 <(dd if=real_build|xxd) <(dd if=failed_build|xxd) |grep '  |	'

127+1 records in
127+1 records out
65535 bytes (66 kB, 64 KiB) copied, 9.3136e-05 s, 704 MB/s
127+1 records in
127+1 records out
65535 bytes (66 kB, 64 KiB) copied, 8.766e-05 s, 748 MB/s
00009030: 9220 4692 209f 9220 ad92 206e c1a9 ff8d  . F. .. .. n....	  |	00009030: 9220 4692 209f 9220 ad92 2063 c1a9 ff8d  . F. .. .. c....
00009140: 96c1 a910 8504 20ad 92ce 0506 101b f8a5  ...... .........	  |	00009140: 89c1 a910 8504 20ad 92ce 0506 101b f8a5  ...... .........
00009160: 0320 fecc a914 8d05 0620 abb0 a918 a404  . ....... ......	  |	00009160: 0320 f0cc a914 8d05 0620 a8b0 a918 a404  . ....... ......
00009190: 8448 adca 6029 0795 4685 9f20 96c1 20c5  .H..`)..F.. .. .	  |	00009190: 8448 adca 6029 0795 4685 9f20 89c1 20c5  .H..`)..F.. .. .
000097b0: 02f0 0320 b5cc a52a 8d00 02a5 2b8d 0102  ... ...*....+...	  |	000097b0: 02f0 0320 a7cc a52a 8d00 02a5 2b8d 0102  ... ...*....+...
00009800: 0130 0160 ad02 02c9 10d0 0320 eecc ad07  .0.`....... ....	  |	00009800: 0130 0160 ad02 02c9 10d0 0320 e

```
0000a8e0: 318d a62f 8da8 2f20 a8aa a901 a000 207f  1../../ ...... .	  |	0000a8e0: 318d a62f 85a8 20a7 aaa9 01a0 0020 7ea9  1../.. ...... ~.
0000a8f0: a924 0530 09a5 4305 4405 45b8 5002 a53e  .$.0..C.D.E.P..>	  |	0000a8f0: 2405 3009 a543 0544 0545 b850 02a5 3ef0  $.0..C.D.E.P..>.
```

This is the following from ALSCOR.LST. THe problem seems to be that the assembler doesn't understand `SCLEVEL+2` properly. It should use an `8D` for the `STA` not an `85` since it's a two byte address.

```ASM
    59   0028    20  0263'                      JSR MSGS
    60   002B    20  025C'                      JSR VGCNTR
    61   002E    AD  0000G                      LDA VGMSGA              ;BLANK OUT LEVEL
    62   0031    8D  0000G                      STA SCLEVEL
    63   0034    85    02G                      STA SCLEVEL+2      <=========== PROBLEM
    64   0036    20  01F7'                      JSR DSPCRD              ;DISPLAY CREDITS & ATARI
    65   0013    25                             ENDIF
    66
     1                                          .SBTTL  INFO DISPLAY-LIVES, SCORES
     2
     3   0039    A9    01                       LDA I,1                 ;(SCALE)
     4   003B    A0    00                       LDY I,0                 ;(PLAYER ID)
     5   003D    20  00CE'                      JSR UPSCLI              ;PLAYER 1 DATA
     6   0040    24    05                       BIT QSTATUS
     7   0042    30    00                       IFPL                    ;ATTRACT?
     8   0044    A5    43                       LDA RSCORL              ;YES. DISPLAY P2 SCORE IF NOT 0
     9   0046    05    44                       ORA RSCORM
    10   0048    05    45                       ORA RSCORH
    11   004A    B8    50    00                 ELSE
         0043    09
```

Let's try fixing this.

In [ ]:
!git clone https://github.com/historicalsource/tempest tempest_modified

In [5]:
!cd tempest_modified && git diff 

diff --git a/ALSCO2.MAC b/ALSCO2.MAC
index 62e9e7b..556c206 100644
--- a/ALSCO2.MAC
+++ b/ALSCO2.MAC
@@ -31,7 +31,7 @@








@@ -70,7 +70,7 @@ INFO:








@@ -1394,4 +1394,4 @@ ZATC1C==ZATC1E-ZATC1S



-                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
\ No newline at end of file

diff --git a/ALSCOR.MAC b/ALSCOR.MAC
index 54dcc19..617b8fc 100644
--- a/ALSCOR.MAC
+++ b/ALSCOR.MAC
@@ -31,7 +31,7 @@








@@ -70,7 +70,7 @@ INFO:








@@ -1394,4 +1394,4 @@ ZATC1C==ZATC1E-ZATC1S



-                                                                                                     

Create a disk with our modified source.

In [1]:
from rt11_utils import create_disk_from_files
create_disk_from_files("./tempest_modified/", "tempest_modified_clean.rk05")

Copy it over to the atari_build folder.

In [3]:
!cp tempest_modified_clean.rk05 ../atari_build/tempest_original.rk05

Now follow the build steps and see what happens. When done copy the disk back over here.

In [4]:
!cp ../atari_build/tempest_original.rk05 tempest_mod_with_object_files.rk05

In [5]:
from rt11_utils import extract_files_from_disk, parse_lda_file
extract_files_from_disk("tempest_mod_with_object_files.rk05", "tempest_mod_object_files")

In [6]:
mod_build = parse_lda_file("tempest_mod_object_files/ALEXEC.LDA")
real_build = parse_lda_file("tempest/ALEXEC.LDA")
open("mod_build",'wb').write(mod_build)
open("real_build",'wb').write(real_build)
!diff -y -W 150 <(dd if=real_build|xxd) <(dd if=mod_build|xxd) |grep '  |	'

127+1 records in
127+1 records out
65535 bytes (66 kB, 64 KiB) copied, 9.3128e-05 s, 704 MB/s
127+1 records in
127+1 records out
65535 bytes (66 kB, 64 KiB) copied, 0.000121154 s, 541 MB/s
00009030: 9220 4692 209f 9220 ad92 206e c1a9 ff8d  . F. .. .. n....	  |	00009030: 9220 4692 209f 9220 ad92 2064 c1a9 ff8d  . F. .. .. d....
00009140: 96c1 a910 8504 20ad 92ce 0506 101b f8a5  ...... .........	  |	00009140: 8ac1 a910 8504 20ad 92ce 0506 101b f8a5  ...... .........
00009160: 0320 fecc a914 8d05 0620 abb0 a918 a404  . ....... ......	  |	00009160: 0320 f1cc a914 8d05 0620 a9b0 a918 a404  . ....... ......
00009190: 8448 adca 6029 0795 4685 9f20 96c1 20c5  .H..`)..F.. .. .	  |	00009190: 8448 adca 6029 0795 4685 9f20 8ac1 20c5  .H..`)..F.. .. .
000097b0: 02f0 0320 b5cc a52a 8d00 02a5 2b8d 0102  ... ...*....+...	  |	000097b0: 02f0 0320 a8cc a52a 8d00 02a5 2b8d 0102  ... ...*....+...
00009800: 0130 0160 ad02 02c9 10d0 0320 eecc ad07  .0.`....... ....	  |	00009800: 0130 0160 ad02 02c9 10d0 0320

0000af10: bdfa 3191 74c8 bdfb 3191 74c8 c638 c639  ..1.t...1.t..8.9	  |	0000af10: b5fa 9174 c8b5 fb91 74c8 c638 c639 10e3  ...t....t..8.9..
0000af20: 10e1 884c 5fdf ad00 060d 0106 f040 a212  ...L_........@..	  |	0000af20: 884c 50df ad00 060d 0106 f040 a212 2014  .LP........@.. .